In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

In [67]:
model_name = "hugohrban/progen2-medium"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

configuration_progen.py:   0%|          | 0.00/2.63k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/hugohrban/progen2-medium:
- configuration_progen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_progen.py:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/hugohrban/progen2-medium:
- modeling_progen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [ ]:
model.eval()

class ZToPrefix(nn.Module):
    def __init__(self, input_dim, prefix_length, d_model):
        super().__init__()
        # A simple linear projection that outputs prefix_length * d_model values
        self.fc = nn.Linear(input_dim, prefix_length * d_model)
        self.prefix_length = prefix_length
        self.d_model = d_model
        
    def forward(self, x):
        # x shape: (batch_size, input_dim)
        batch_size = x.size(0)
        prefix = self.fc(x)            # Shape: (batch_size, prefix_length * d_model)
        prefix = prefix.view(batch_size, self.prefix_length, self.d_model)  # Reshape to (batch_size, prefix_length, d_model)
        return prefix

In [12]:
batch_size = 1
esm_dim = 768  # Adjust this to match the dimension of your ESM representations
generated_esm_embedding = torch.randn(batch_size, esm_dim)

In [32]:
prefix_length = 10
d_model = model.config.embed_dim  # e.g., 1024 or whatever the model uses

prefix_module = ESMToPrefix(input_dim=esm_dim, prefix_length=prefix_length, d_model=d_model)

# Map the generated ESM embedding to prefix embeddings.
prefix_embeddings = prefix_module(generated_esm_embedding)  # shape: (batch_size, prefix_length, d_model)



# forward pass
logits = model(inputs_embeds=prefix_embeddings).logits

print(logits.shape)
# print output probabilities
next_token_logits = logits[:, -1, :]
next_token_probs = F.softmax(next_token_logits, dim=-1)
print(next_token_logits.shape)
for i in range(tokenizer.vocab_size):
    print(tokenizer.decode(i), next_token_probs[0, i].item())


torch.Size([1, 10, 32])
torch.Size([1, 32])
<|pad|> 9.670131449063257e-13
<|bos|> 9.261969446602847e-13
<|eos|> 1.0327710041335703e-11
1 0.1502617597579956
2 0.31953683495521545
A 0.018371088430285454
B 5.238836138232728e-07
C 0.0053237020038068295
D 0.01670844480395317
E 0.028254467993974686
F 0.009900936856865883
G 0.03264149650931358
H 0.01732638105750084
I 0.015535769984126091
K 0.023146817460656166
L 0.01947997882962227
M 0.15795548260211945
N 0.024909188970923424
O 6.143419284398988e-08
P 0.011583761312067509
Q 0.02621517702937126
R 0.02283564582467079
S 0.026994973421096802
T 0.02846738137304783
U 6.175801559038518e-07
V 0.020205963402986526
W 0.004934186115860939
X 0.003107278374955058
Y 0.016301607713103294
Z 5.45181023881014e-07


In [69]:
prompt = "1MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
input_ids = torch.tensor(tokenizer.encode(prompt)).to(model.device)

# forward pass
logits = model(input_ids).logits

In [78]:
# for i in range(len(prompt)):

#     print(tokenizer.decode(F.softmax(logits[i, :], dim=-1).argmax()), prompt[i+1])
tokenizer.encode('<|bos|>')

[1]

In [ ]:
class ZToPrefix(nn.Module):
    def __init__(self, input_dim, prefix_length, d_model):
        super().__init__()
        # A simple linear projection that outputs prefix_length * d_model values
        self.fc = nn.Linear(input_dim, prefix_length * d_model)
        self.prefix_length = prefix_length
        self.d_model = d_model
        
    def forward(self, x):
        # x shape: (batch_size, input_dim)
        batch_size = x.size(0)
        prefix = self.fc(x)            # Shape: (batch_size, prefix_length * d_model)
        prefix = prefix.view(batch_size, self.prefix_length, self.d_model)  # Reshape to (batch_size, prefix_length, d_model)
        return prefix

class ConditionedProgen2(nn.Module):
    def __init__(self, progen2_name='hugohrban/progen2-medium', 
                 latent_dim=128, prefix_length=10, ):
        super().__init__()
        self.model = AutoModelForCausalLM.from_pretrained(progen2_name, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(progen2_name, trust_remote_code=True)
        self.prefix_module = ZToPrefix(latent_dim, prefix_length, self.model.config.embed_dim)

    def forward(self, input_ids, esm_embedding):
        # Get the prefix embeddings from the ESM embedding
        prefix_embeddings = self.prefix_module(esm_embedding)
        
        # Concatenate the prefix embeddings with the input embeddings
        input_embeddings = self.model.transformer.wte(input_ids)  # Get the token embeddings

        combined_embeddings = torch.cat((prefix_embeddings, input_embeddings), dim=1)
        
        # Forward pass through the model
        outputs = self.model(inputs_embeds=combined_embeddings)
        return outputs.logits[:, 1:, :]
        
class Progen2Generator:
    def __init__(
        self,
        progen2_name="hugohrban/progen2-medium",
        latent_dim=128,
        prefix_length=10,
        temperature=1.0,
        max_length=1024,
        device="cuda",
    ):
        self.model = ConditionedProgen2(
            progen2_name=progen2_name,
            latent_dim=latent_dim,
            prefix_length=prefix_length,
        ).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(progen2_name, trust_remote_code=True)
        self.temperature = temperature
        self.max_length = max_length
        self.device = device

    def loss(self, x, latent):
        input_ids = self.tokenizer.encode(x.to(self.device))
        shift_logits = self.model(input_ids, latent)[:, :-1, :]
        shift_labels = input_ids[:, 1:]
        loss = F.cross_entropy(
            shift_logits.reshape(-1, shift_logits.size(-1)),
            shift_labels.reshape(-1)
        )
        return loss

    def sample(self, latent, num_samples=1, return_seq=False):
        device = latent.device
        batch_size = latent.size(0)
        start_ids = torch.tensor(
            [self.tokenizer.encode('1')] * batch_size,
            device=device
        )

        all_samples = []
        for _ in range(num_samples):
            with torch.no_grad():
                out = self._generate(start_ids, latent)
            all_samples.append(out)

        out = torch.stack(all_samples, dim=1)

        if return_seq:
            texts = [
                [self.tokenizer.decode(out[b, n], skip_special_tokens=True)
                 for n in range(num_samples)]
                for b in range(batch_size)
            ]
            return out, texts

        return out

    def _generate(self, input_ids, latent):
        cur_ids = input_ids
        for _ in range(self.max_length - 2):
            with torch.no_grad():
                logits = self.model(cur_ids, latent)
            next_logits = logits[:, -1, :] / self.temperature
            probs = F.softmax(next_logits, dim=-1)
            next_token = torch.multinomial(probs, 1)
            cur_ids = torch.cat([cur_ids, next_token], dim=1)
        return cur_ids

In [36]:
pg2 = Progen2Generator(max_length=512)

In [37]:
pg2.sample(torch.rand(1, 128).to('cuda'), num_samples=1, return_seq=True)

(tensor([[[ 3, 12, 23, 25, 15, 10, 11, 11, 14, 23, 21, 28, 20, 11, 22, 25, 20,
           15, 19, 28, 11, 10, 28, 23, 14, 22, 20, 21, 25,  5, 10, 21, 10,  8,
           17,  9, 15, 20, 21, 26, 14,  5, 11, 11, 14,  5,  5, 20,  9, 25, 21,
           16, 25,  9, 10, 11,  7,  5,  8, 12,  9, 19,  9,  5, 26, 23, 14, 25,
            5, 11,  5,  9, 11, 12,  9, 21, 25, 20, 15,  7, 22,  8, 21, 21, 22,
           12, 25, 10, 16,  8, 16, 23, 11, 11, 25, 17, 15, 25, 28,  5, 25, 25,
           11,  5, 17,  9, 23, 23, 28, 22, 15, 28, 25, 11, 15,  9, 14, 25, 28,
            5, 12,  5,  8,  5, 26, 17, 23, 21, 28, 15, 21, 11,  5, 14, 20, 11,
           25, 22, 15, 21,  8, 25, 28,  8,  8, 14, 21, 21, 23, 28, 17, 25, 22,
           13, 12, 21, 15, 22, 25, 21, 28,  7, 14,  8,  5, 23,  5,  5,  9,  7,
           14, 23, 25, 21, 10, 16,  5, 13,  5, 13, 23, 17, 11, 21, 12, 12, 19,
           21, 13, 12, 15, 12,  9, 28, 12, 13, 10, 17, 22, 26, 15, 26, 21, 17,
            8, 15, 20, 21, 15,  5, 14,  5, 11, 22, 1

In [62]:
from transformers import EsmModel, EsmConfig
from encoder.encoders import DistributionEncoder

class ESMFeatureExtractor(nn.Module):
    def __init__(self, esm_model_name="facebook/esm2_t6_8M_UR50D", output_dim=320, pooling="mean", freeze=False):
        super().__init__()
        self.esm = EsmModel.from_pretrained(esm_model_name)
        if freeze:
            for p in self.esm.parameters(): p.requires_grad = False
        self.pooling = pooling
        h = self.esm.config.hidden_size
        self.proj = nn.Linear(h, output_dim) if output_dim != h else nn.Identity()

    def forward(self, input_ids, attention_mask=None):
        x = self.esm(input_ids, attention_mask=attention_mask).last_hidden_state
        if self.pooling == "cls":
            pooled = x[:, 0]
        elif self.pooling == "mean":
            mask = attention_mask.unsqueeze(-1).float()
            pooled = (x * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        elif self.pooling == "max":
            mask = attention_mask.unsqueeze(-1).float()
            x[mask == 0] = -1e9
            pooled = x.max(1).values
        else:
            raise ValueError("bad pooling :(")
        return self.proj(pooled)

class ProteinSetEncoder(nn.Module):
    def __init__(self, esm_model_name="facebook/esm2_t6_8M_UR50D", esm_dim=320, latent_dim=32, hidden_dim=128,
                 pooling="cls", freeze=False, dist_type="tx", layers=2, heads=4):
        super().__init__()
        self.esm_extractor = ESMFeatureExtractor(esm_model_name, esm_dim, pooling, freeze)
        if dist_type == "tx":
            from encoder.encoders import DistributionEncoderTx as DE
            self.dist = DE(esm_dim, latent_dim, hidden_dim, None, layers, heads)
        elif dist_type == "gnn":
            from encoder.encoders import DistributionEncoderGNN as DE
            self.dist = DE(esm_dim, latent_dim, hidden_dim, None, layers, fc_layers=2)
        elif dist_type == "median_gnn":
            from encoder.encoders import DistributionEncoderMedianGNN as DE
            self.dist = DE(esm_dim, latent_dim, hidden_dim, None, layers, fc_layers=2)
        else:
            raise ValueError("bad dist type :(")

    def forward(self, samples):
        b, s = samples['input_ids'].shape[:2]
        ids = samples['input_ids'].view(b * s, -1)
        mask = samples['attention_mask'].view(b * s, -1)
        feats = self.esm_extractor(ids, mask).view(b, s, -1)
        return self.dist(feats)

In [63]:
enc = ProteinSetEncoder()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
esm_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D", trust_remote_code=True)

In [81]:
seq1 = '1MSKGEELF2'
seq2 = '1TGVVPILV2'

seqs= [seq2]*10

samples = esm_tokenizer(seqs, return_tensors="pt", padding=True, truncation=True)

samples['input_ids'] = samples['input_ids'].unsqueeze(0)